Future self, move notebook and artnames.txt to outermost directory.

In [1]:
import py7zr as pz
import requests
from bs4 import BeautifulSoup
from src.libcode import list_to_txt, txt_to_list
import io

In [2]:
def download_unzip(link):
    resp = requests.get(i, stream=True)
    z = pz.SevenZipFile(io.BytesIO(resp.content))
    z.extractall("src/data")
#     with pz.SevenZipFile('src/data/' + fname, mode='r') as z:
#         z.extractall("src/data")


In [3]:
def scrape_links(fname="artnames.txt"):
    resp = requests.get("https://dumps.wikimedia.org/enwiki/20210101/")
    dump_soup = BeautifulSoup(resp.text)
    dumps_base = "https://dumps.wikimedia.org"
    link_list = []
    for i in dump_soup.find_all("a"):
        if "pages-meta-history" in i.text and ".7z" in i.text:
            link_list.append(dumps_base + i["href"])
    list_to_txt(fname, link_list)
    
def retrieve_links(fname = "artnames.txt"):
    return txt_to_list(fname)

In [4]:
link_list = retrieve_links()

In [5]:
link_list[0:5]

['https://dumps.wikimedia.org/enwiki/20210101/enwiki-20210101-pages-meta-history1.xml-p1p873.7z',
 'https://dumps.wikimedia.org/enwiki/20210101/enwiki-20210101-pages-meta-history1.xml-p874p1816.7z',
 'https://dumps.wikimedia.org/enwiki/20210101/enwiki-20210101-pages-meta-history1.xml-p1817p2753.7z',
 'https://dumps.wikimedia.org/enwiki/20210101/enwiki-20210101-pages-meta-history1.xml-p2754p3454.7z',
 'https://dumps.wikimedia.org/enwiki/20210101/enwiki-20210101-pages-meta-history1.xml-p3455p4320.7z']

In [6]:
len(link_list)

689

In [7]:
download_unzip(link_list[0])

NameError: name 'i' is not defined

In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import sys
from matplotlib import pyplot as plt
import requests
from bs4 import BeautifulSoup

In [9]:
base_url = 'https://dumps.wikimedia.org/enwiki/'
index = requests.get(base_url).text
soup_index = BeautifulSoup(index, 'html.parser')
# Find the links on the page
dumps = [a['href'] for a in soup_index.find_all('a') if 
         a.has_attr('href')]
dumps

['../',
 '20201020/',
 '20201101/',
 '20201120/',
 '20201201/',
 '20201220/',
 '20210101/',
 '20210120/',
 'latest/']

In [10]:
if "latest" not in dumps[-1]:
    print("Couldn't find the latest dump")
    print(dumps[-1])
    print(dumps[-2])
    exit

In [11]:
dump_url = base_url + '20201020/'
# Retrieve the html
dump_html = requests.get(dump_url).text
# Convert to a soup
soup_dump = BeautifulSoup(dump_html, 'html.parser')
# Find list elements with the class file
soup_dump.find_all('li', {'class': 'file'})[:3]

[<li class="file"><a href="/enwiki/20201020/enwiki-20201020-pages-articles-multistream.xml.bz2">enwiki-20201020-pages-articles-multistream.xml.bz2</a> 17.6 GB</li>,
 <li class="file"><a href="/enwiki/20201020/enwiki-20201020-pages-articles-multistream-index.txt.bz2">enwiki-20201020-pages-articles-multistream-index.txt.bz2</a> 216.3 MB</li>,
 <li class="file"><a href="/enwiki/20201020/enwiki-20201020-pages-articles-multistream1.xml-p1p41242.bz2">enwiki-20201020-pages-articles-multistream1.xml-p1p41242.bz2</a> 232.1 MB</li>]

Types of files
* pages-articles-multistream.xml.bz2 – Current revisions only, no talk or user pages; this is probably what you want, and is approximately 18 GB compressed (expands to over 78 GB when decompressed).
* pages-meta-current.xml.bz2 – Current revisions only, all pages (including talk)
* abstract.xml.gz – page abstracts
* all-titles-in-ns0.gz – Article titles only (with redirects)
* SQL files for the pages and links are also available

In [19]:
# Search for "current" files
files = []
for file in soup_dump.find_all("li", {"class": "file"}):
    text = file.text
    #print(file.text)
    # what are these?
    if "pages-meta-current" in text:
        files.append((text.split()[0], text.split()[1:]))
files_to_download = [file[0] for file in files]
print(len(files_to_download))
files[0]

60


('enwiki-20201020-pages-meta-current.xml.bz2', ['32.1', 'GB'])